In [1]:
import pandas as pd
import pickle

In [2]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codebert_embeddings/df_fe_embeded_by_line_codebert_base.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_35919/711735428.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [3]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,12150,critical,public SystemDiagnosticsDTO createSystemDi...,1,"[[-10.351140975952148, 18.46828842163086, -1.5...","[[-0.16430382430553436, 0.2931474447250366, -0..."
1,10168,critical,protected void addRequiredAlertProperties(Se...,1,"[[-0.611336886882782, 1.667067050933838, -0.08...","[[-0.12226738035678864, 0.33341342210769653, -..."
2,12783,critical,"@SuppressWarnings(""try"")\n private void...",1,"[[-25.163928985595703, 32.38447189331055, -5.0...","[[-0.20458479225635529, 0.26328837871551514, -..."
3,12754,critical,public void validateDepositDetailForUpdate...,1,"[[-13.66582202911377, 26.740516662597656, -6.1...","[[-0.12201626598834991, 0.23875461518764496, -..."
4,4277,critical,@Override\n public ListenableFuture<Voi...,1,"[[-15.209061622619629, 20.849018096923828, -4....","[[-0.19752028584480286, 0.2707664668560028, -0..."


In [4]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,12150,critical,1,"[[-10.351140975952148, 18.46828842163086, -1.5..."
1,10168,critical,1,"[[-0.611336886882782, 1.667067050933838, -0.08..."
2,12783,critical,1,"[[-25.163928985595703, 32.38447189331055, -5.0..."
3,12754,critical,1,"[[-13.66582202911377, 26.740516662597656, -6.1..."
4,4277,critical,1,"[[-15.209061622619629, 20.849018096923828, -4...."


In [5]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    [[-10.351140975952148, 18.46828842163086, -1.5...
1    [[-0.611336886882782, 1.667067050933838, -0.08...
2    [[-25.163928985595703, 32.38447189331055, -5.0...
3    [[-13.66582202911377, 26.740516662597656, -6.1...
4    [[-15.209061622619629, 20.849018096923828, -4....
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'list'>    3027
Name: count, dtype: int64


In [6]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [7]:
print(embeded_sum["embeded_sequence_sum"].head())

0    [[-10.351140975952148, 18.46828842163086, -1.5...
1    [[-0.611336886882782, 1.667067050933838, -0.08...
2    [[-25.163928985595703, 32.38447189331055, -5.0...
3    [[-13.66582202911377, 26.740516662597656, -6.1...
4    [[-15.209061622619629, 20.849018096923828, -4....
Name: embeded_sequence_sum, dtype: object


In [8]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [9]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [10]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [11]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 2894


In [12]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [-10.351140975952148, 18.46828842163086, -1.58...
1    [-0.611336886882782, 1.667067050933838, -0.084...
2    [-25.163928985595703, 32.38447189331055, -5.07...
3    [-13.66582202911377, 26.740516662597656, -6.12...
4    [-15.209061622619629, 20.849018096923828, -4.4...
Name: embeded_sequence_sum, dtype: object


In [13]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3027
Name: count, dtype: int64


In [14]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


       emb_1      emb_2     emb_3      emb_4      emb_5      emb_6      emb_7  \
0 -10.351141  18.468288 -1.587198   5.859317   6.370951 -15.148269   4.167015   
1  -0.611337   1.667067 -0.084088   0.345362   0.262180  -0.930403   0.067351   
2 -25.163929  32.384472 -5.075072  14.601456  21.233459 -35.906075  11.761786   
3 -13.665822  26.740517 -6.125227  11.503922  20.571180 -28.369169   9.178022   
4 -15.209062  20.849018 -4.461039   6.608896  12.249491 -21.742653   7.111282   

      emb_8      emb_9     emb_10  ...    emb_762    emb_763   emb_764  \
0  1.770694  16.307508 -18.527201  ...  40.539562  -5.868604 -0.525147   
1  0.382352   1.096432  -1.287243  ...   2.431135  -0.560943  0.054394   
2  4.701000  33.779083 -38.444489  ...  89.350410  -8.292704 -4.148782   
3  3.348720  27.385149 -33.969685  ...  81.852448 -11.761877  0.143565   
4  2.520332  18.878557 -22.469709  ...  51.909943  -5.237012 -1.983984   

     emb_765    emb_766    emb_767    emb_768  label  sample_id  sev

In [15]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [16]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    [[-0.16430382430553436, 0.2931474447250366, -0...
1    [[-0.12226738035678864, 0.33341342210769653, -...
2    [[-0.20458479225635529, 0.26328837871551514, -...
3    [[-0.12201626598834991, 0.23875461518764496, -...
4    [[-0.19752028584480286, 0.2707664668560028, -0...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'list'>    3027
Name: count, dtype: int64


In [17]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [18]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 16292


In [19]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [20]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 2289


In [21]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [-0.16430382430553436, 0.2931474447250366, -0....
1    [-0.12226738035678864, 0.33341342210769653, -0...
2    [-0.20458479225635529, 0.26328837871551514, -0...
3    [-0.12201626598834991, 0.23875461518764496, -0...
4    [-0.19752028584480286, 0.2707664668560028, -0....
Name: embeded_sequence_avg, dtype: object


In [22]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3027
Name: count, dtype: int64


In [23]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.164304,0.293147,-0.025194,0.093005,0.101126,-0.240449,0.066143,0.028106,0.258849,-0.294083,...,0.643485,-0.093152,-0.008336,0.464331,-0.296749,-0.385003,0.395283,1,12150,critical
1,-0.122267,0.333413,-0.016818,0.069072,0.052436,-0.186081,0.013470,0.076470,0.219286,-0.257449,...,0.486227,-0.112189,0.010879,0.412307,-0.353872,-0.348544,0.363493,1,10168,critical
2,-0.204585,0.263288,-0.041261,0.118711,0.172630,-0.291919,0.095624,0.038220,0.274627,-0.312557,...,0.726426,-0.067420,-0.033730,0.540107,-0.246627,-0.438533,0.391139,1,12783,critical
3,-0.122016,0.238755,-0.054690,0.102714,0.183671,-0.253296,0.081947,0.029899,0.244510,-0.303301,...,0.730825,-0.105017,0.001282,0.462068,-0.248525,-0.410063,0.402719,1,12754,critical
4,-0.197520,0.270766,-0.057936,0.085830,0.159084,-0.282372,0.092354,0.032732,0.245176,-0.291814,...,0.674155,-0.068013,-0.025766,0.483451,-0.264792,-0.420454,0.388313,1,4277,critical


In [24]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)